In [1]:
from ape import networks, Contract
import pandas as pd

from helpers.coingecko import get_prices
from helpers.subgraph_endpoints import *
from helpers.aura import get_gravi_in_balancer_pool, weekly_emissions_after_fee, aura_mint_ratio
from helpers.vp_info import get_council_vp_fee, get_voter_vp
from helpers.bribe_info import MAX_BRIBE, GAUGE_CAP
from helpers.formatter import pct_format, dollar_format

In [ ]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

In [3]:
bal_price, aura_price,  badger_price, rpl_price = get_prices()
data = [["${:.2f}".format(badger_price),  "${:.2f}".format(bal_price), "${:.2f}".format(aura_price), "${:.2f}".format(rpl_price)]]
headers = ["Badger price", "Balancer price", "Aura price", "Rpl price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,Badger price,Balancer price,Aura price,Rpl price
0,$2.26,$5.56,$1.89,$19.20


In [5]:
# ping a current block, choose: https://etherscan.io/block/16237419
block_current_proposal = 16237419
last_proposal_id = "0xee3d9361d2e32dfa569baaed775575691df8163fbcc8ab936afe4d433535ab15"

# check active voter in past proposal as ref
response_last_proposal = get_proposal_info(last_proposal_id)

aura_voting_actively_last_round = response_last_proposal["scores_total"]

# value from https://hiddenhand.finance/balancer
cost_per_vote_vebal = 0.08

data = [[block_current_proposal,  f'{aura_voting_actively_last_round:.0f}', dollar_format(cost_per_vote_vebal)]]
headers = ["Proposal block height", "Past Round Active vlAURA votes", "Cost per Vote Last Round"]
df_llama = pd.DataFrame(data, columns=headers)
df_llama


,Proposal block height,Past Round Active vlAURA votes,Cost per Vote Last Round
0,16237419,9773627,$0.0800


In [6]:
# emissions ecosystem
aura = Contract('0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF')
aura_mint_ratio = aura_mint_ratio(aura, block_current_proposal)
weekly_emissions_after_fee = weekly_emissions_after_fee(aura_mint_ratio, bal_price, aura_price)
biweekly_emissions = weekly_emissions_after_fee * 2
data = [[aura_mint_ratio, dollar_format(biweekly_emissions)]]
headers = ["Aura Mint Balancer Ratio", "Bi-weekly Emissions"]
df_emissions = pd.DataFrame(data, columns=headers)
df_emissions

,Aura Mint Balancer Ratio,Bi-weekly Emissions
0,3.615009,$2695339.7023


In [8]:
vlAURA = Contract("0x3Fa73f1E5d8A792C80F426fc8F84FBF7Ce9bBCAC")
vlAURA_ts = vlAURA.totalSupply(block_identifier=block_current_proposal)/1e18
council_fee = get_council_vp_fee(vlAURA, block_current_proposal)

# digg pool
balancer_vault = Contract("0xBA12222222228d8Ba445958a75a0704d566BF2C8")
gravi_in_digg_pool = get_gravi_in_balancer_pool(balancer_vault, block_current_proposal)
total_gravi_vp_for_digg = gravi_in_digg_pool

treasury_votes = get_voter_vp(vlAURA, block_current_proposal)
total_vp_badgerdao = treasury_votes + council_fee

pct_controlled_aura_by_badger_treasury = treasury_votes / vlAURA_ts
pct_controlled_aura_by_badger_treasury_active = treasury_votes / aura_voting_actively_last_round
pct_controlled_aura_by_badger = total_vp_badgerdao/ vlAURA_ts
pct_controlled_aura_by_badger_active = total_vp_badgerdao/ aura_voting_actively_last_round
pct_controlled_aura_by_council_active = total_gravi_vp_for_digg / aura_voting_actively_last_round
data = [[treasury_votes, council_fee, total_vp_badgerdao, gravi_in_digg_pool, pct_format(pct_controlled_aura_by_badger), pct_format(pct_controlled_aura_by_badger_active)]]
headers = ["Treasury VP", "Council Fee", "Total Badger VP", "Gravi in Digg pool", "Badger Aura Controlled - everyone Votes", "Badger Aura Controlled - proactive voters"]
df_vp = pd.DataFrame(data, columns=headers)
df_vp

,Treasury VP,Council Fee,Total Badger VP,Gravi in Digg pool,Badger Aura Controlled - everyone Votes,Badger Aura Controlled - proactive voters
0,273130.560991,74163.468435,347294.029427,241495.88732,3.180%,3.553%


In [9]:
vebal = Contract("0xC128a9954e6c874eA3d62ce62B468bA073093F25")
vebal_ts = vebal.totalSupplyAt(block_current_proposal) / 1e18
vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", block_current_proposal)/1e18
pct_aura_vebal_controlled = vebal_aura_controlled/vebal_ts
pct_badger_vebal_controlled = pct_controlled_aura_by_badger*pct_aura_vebal_controlled
pct_badger_vebal_controlled_active = pct_controlled_aura_by_badger_active*pct_aura_vebal_controlled
pct_council_vebal_controlled_active = pct_controlled_aura_by_council_active * pct_aura_vebal_controlled
data = [[pct_format(pct_aura_vebal_controlled), pct_format(pct_badger_vebal_controlled), pct_format(pct_badger_vebal_controlled_active)]]
headers = ["Aura veBAL controlled", "Badger veBAL controlled - everyone votes", "Badger veBAL controlled - proactive voters"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Aura veBAL controlled,Badger veBAL controlled - everyone votes,Badger veBAL controlled - proactive voters
0,22.656%,0.721%,0.805%


In [10]:
# NOTE: block breakdown for badger/wbtc max cap
vebal_controlled = vebal_ts * pct_badger_vebal_controlled_active
vebal_max_cap = vebal_ts * (GAUGE_CAP)
vebal_required = vebal_max_cap - vebal_controlled
cost_max_cap = vebal_required * cost_per_vote_vebal
badger_to_bribe = cost_max_cap / badger_price
data = [[vebal_max_cap, vebal_controlled, dollar_format(cost_max_cap), badger_to_bribe]]
headers = ["veBAL badger/wbtc pool Max Cap", "veBAL BADGER controlled", "$ Cost of bribe", "Badger Bribe Max Cap"]
df_bribes_badger_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_pool

,veBAL badger/wbtc pool Max Cap,veBAL BADGER controlled,$ Cost of bribe,Badger Bribe Max Cap
0,235567.129449,94822.717483,$11259.5530,4982.103078


In [11]:
# NOTE: remaining bribes budget to reallocate
remaining_bribe = MAX_BRIBE - badger_to_bribe
data = [[badger_to_bribe, remaining_bribe]]
headers = ["Bribe max cap wbtc/badger", "Remaining bribe reth/badger"]
df_bribes_split = pd.DataFrame(data, columns=headers)
df_bribes_split

,Bribe max cap wbtc/badger,Remaining bribe reth/badger
0,4982.103078,11017.896922


In [12]:
# NOTE: block breakdown for badger/reth remaining bribe
cost_of_remaining_bribe = remaining_bribe * badger_price
vebal_vp_bought = cost_of_remaining_bribe / cost_per_vote_vebal
vebal_pct_weight_bought = vebal_vp_bought / vebal_ts 
data = [[cost_of_remaining_bribe, pct_format(vebal_pct_weight_bought)]]
headers = ["$ Cost of Bribe", "veBAL Bribe Bought"]
df_bribes_badger_reth_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_reth_pool

,$ Cost of Bribe,veBAL Bribe Bought
0,24900.447043,2.643%


In [14]:
# treasury expected earnings bi-weekly expectation in BADGER/WBTC pool
tvl_badger = get_tvl_balancer_badger_pool()
badger_pool = Contract("0xb460DAa847c45f1C4a41cb05BFB3b51c92e41B36")
pool_supply = badger_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0xAF50825B010Ae4839Ac444f6c12D44b96819739B")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_badger * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_badger
pool_earnings = biweekly_emissions * GAUGE_CAP
treasury_roi = (pool_earnings*pool_capture) / cost_max_cap
data = [[dollar_format(pool_earnings), dollar_format(tvl_badger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "BADGER Pool TVL", "BADGER Pool Capture", "Treasury yield earned (USD) - biweekly", "Treasury ROI - biweekly"]
df_earnings_badger_wbtc_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_wbtc_pool

,Pool yield (USD) - biweekly,BADGER Pool TVL,BADGER Pool Capture,Treasury yield earned (USD) - biweekly,Treasury ROI - biweekly
0,$53906.7940,$6387799.6959,30.203%,$16281.2817,144.600%


In [15]:
# treasury expected earnings bi-weekly expectation in BADGER/RETH pool
tvl_rethbadger = get_tvl_balancer_badgerreth_pool()
badgereth_pool = Contract("0xe340EBfcAA544da8bB1Ee9005F1a346D50Ec422e")
pool_badgereth_supply = badgereth_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0xcAb1921E4fc9f4C26B877B58D9713CD19E9eADD1")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_rethbadger * (vault_rewards_contract_balance/pool_badgereth_supply)
pool_capture = vault_owned_tcl / tvl_rethbadger
pool_earnings = biweekly_emissions * vebal_pct_weight_bought
treasury_roi = (pool_earnings*pool_capture) / cost_of_remaining_bribe
data = [[dollar_format(pool_earnings), dollar_format(tvl_rethbadger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "BADGER Pool TVL", "BADGER Pool Capture", "Treasury yield earned (USD) - biweekly", "Treasury ROI - biweekly"]
df_earnings_badger_reth_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_reth_pool

,Pool yield (USD) - biweekly,BADGER Pool TVL,BADGER Pool Capture,Treasury yield earned (USD) - biweekly,Treasury ROI - biweekly
0,$71227.2163,$10640943.6504,13.526%,$9634.4047,38.692%


In [46]:
# in which capture starts to become breakeven on the expense of the bribe?
break_even_capture = cost_of_remaining_bribe / pool_earnings
data = [[pct_format(break_even_capture)]]
headers = ["BADGER Breakeven Pool Capture"]
df_breakeven_capture_badger_reth_pool = pd.DataFrame(data, columns=headers)
df_breakeven_capture_badger_reth_pool

,BADGER Breakeven Pool Capture
0,30.336%
